In [24]:
!pip install reportlab
!pip install matplotlib networkx
!pip install pyttsx3


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   -------------------------------

In [30]:
# === Imports & Setup ===
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import tempfile
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import matplotlib.pyplot as plt
import networkx as nx
import uuid



* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [31]:
# Load API key
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)



In [32]:
# === Gemini Caller ===
def call_gemini(system_prompt, user_prompt, language, temperature=0.5, max_output=400):
    model = genai.GenerativeModel("models/gemini-1.5-flash")
    user_prompt = f"{user_prompt}\n\nRespond in {language}."
    response = model.generate_content(
        [system_prompt, user_prompt],
        generation_config={"temperature": temperature, "max_output_tokens": max_output},
        stream=True
    )
    full_text = ""
    for chunk in response:
        if chunk.text:
            full_text += chunk.text
            yield full_text
    yield full_text



In [ ]:
# === Agents ===
def run_ceo(problem, constraints, language, temp):
    return call_gemini("You are CEO Agent. Generate concise product ideas.",
                       f"Problem: {problem}\nConstraints: {constraints}", language, temp)

def run_researcher(idea, language, temp):
    return call_gemini("You are Research Agent. Provide background and competitors.",
                       f"Idea to research: {idea}", language, temp)

def run_coder(idea, research, language, temp):
    return call_gemini("You are Coder Agent. Suggest an implementation approach.",
                       f"Idea: {idea}\nResearch: {research}", language, temp)

def run_critic(idea, research, codeplan, language, temp):
    return call_gemini("You are Critic Agent. Analyze risks and flaws.",
                       f"Idea: {idea}\nResearch: {research}\nCode Plan: {codeplan}", language, temp)

